In [16]:

import os
import torch
import numpy as np
import pytorch_lightning as pl
from torch_geometric.datasets import Planetoid
from torch_geometric.data import Data
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from lightning.pytorch import loggers as pl_loggers
import torch_geometric.transforms as T
import torch_geometric.data as geom_data
from torch.utils.tensorboard import SummaryWriter

%reload_ext autoreload
%autoreload 2

#### Configure device

In [17]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if device != "cpu":
    torch.set_float32_matmul_precision('high')
    
# device = "cpu"
# torch.set_float32_matmul_precision("high")

num_workers = os.cpu_count()
device, num_workers

(device(type='cuda', index=0), 12)

### Directory configuration and load data

In [18]:
cwd = os.getcwd()
tb_logging_dir = os.path.join(cwd, "lightning_logs")
exp_name = "AG-GraphSAGE-transductive"
exp_dir = os.path.join(tb_logging_dir, exp_name)
dataset_dir = os.path.join(cwd, "dataset", "CUHKSZ_AcademicGraph")
if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)
    
from utils.dataset.CUHKSZ_AcademicGraph import CUHKSZ_AcademicGraph
AGDataset = CUHKSZ_AcademicGraph(dataset_dir, with_title=True, with_label=True)

AGDataset[0]

D:\GitHub\GNN-Cora-CUHKSZAG\dataset\CUHKSZ_AcademicGraph\raw\CUHKSZ_AcademicGraph_Rawdata.zip
D:\GitHub\GNN-Cora-CUHKSZAG\dataset\CUHKSZ_AcademicGraph\raw\CUHKSZ_AcademicGraph-rawdata_released


Data(x=[6614, 768], edge_index=[2, 12330], y=[6614], title=[6614], train_mask=[6614], val_mask=[6614], test_mask=[6614])

#### Note that if numebr of epoch is large, the kernel will stuck after training! Need to load the saved model mannually!

In [19]:
from utils.model.RS_GraphSAGE import GraphSage

dataset = AGDataset

early_stop_callback = EarlyStopping(
    monitor='val_loss',
    min_delta=0.00,
    patience=10,
    verbose=False,
    mode='min'
)
hparams = {"aggregator_type": 'mean',
           "HIDDEN_DIM": [112, 16],  # size of the embedding
           "BATCH_SIZE": 64,
           "LEARNING_RATE": 0.001,
           "NUM_NEIGHBORS": [10, 10]
           }  # The number of neighbors in each order of sampling

tb_logger = pl_loggers.TensorBoardLogger(tb_logging_dir, name=exp_name)
 
trainer = pl.Trainer(max_epochs=100,
                     callbacks=[early_stop_callback],
                     logger=tb_logger,
                     log_every_n_steps=1,
                     # accelerator="cpu"
                    # num_sanity_val_steps = 0
                     )

version_dir = os.path.join(
    exp_dir, "version_"+str(trainer.logger.version))
writer_acc = SummaryWriter(log_dir=version_dir)
writer_loss = SummaryWriter(log_dir=version_dir)

checkpoint_dir = os.path.join(version_dir, "checkpoints")
print("Saving checkpoints to", checkpoint_dir)

SAGEmodel = GraphSage(dataset=dataset, input_dim=dataset.num_features, hparams=hparams,
                      writer_acc=writer_acc, writer_loss=writer_loss).to(device)
SAGEmodel.data_processing_transductive() 

trainer.fit(SAGEmodel)
trainer.test(SAGEmodel)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Saving checkpoints to D:\GitHub\GNN-Cora-CUHKSZAG\lightning_logs\AG-GraphSAGE-transductive\version_1\checkpoints


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type              | Params
--------------------------------------------
0 | aggr  | MeanAggregation   | 0     
1 | model | Sequential_313bbb | 175 K 
--------------------------------------------
175 K     Trainable params
0         Non-trainable params
175 K     Total params
0.703     Total estimated model params size (MB)


Data Processing Done on: cuda:0


Sanity Checking: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
D:\Anaconda\envs\torchGNN\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9837986846898991     │
│         test_loss         │    0.5497453212738037     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.5497453212738037, 'test_acc': 0.9837986846898991}]

#### Upload trained model

In [4]:
from utils.model.RS_GraphSAGE import GraphSage
hparams = {"aggregator_type": 'mean',
           "HIDDEN_DIM": [112, 16],  # size of the embedding
           "BATCH_SIZE": 64,
           "LEARNING_RATE": 0.001,
           "NUM_NEIGHBORS": [10, 10]
           }
dataset = AGDataset
checkpoint_dir = "lightning_logs\\RS_AG_GraphSAGE_transductive\\best\\checkpoints"
checkpoint_file = os.path.join(checkpoint_dir, os.listdir(checkpoint_dir)[0])
Loaded_model = GraphSage.load_from_checkpoint(checkpoint_file, dataset=dataset, input_dim=dataset.num_features, hparams=hparams)
Loaded_model.data_processing_transductive()

Data Processing Done on: cpu


In [25]:
Loaded_model = SAGEmodel

#### To see the mini-batch's number of negative edges:

In [7]:
tmpLoader = Loaded_model.train_dataloader()
sample = next(iter(tmpLoader))
sample

Data(x=[4277, 768], edge_index=[2, 6750], edge_label=[6750], train_mask=[4277], n_id=[4277], e_id=[6750], input_id=[64], batch_size=64)

#### To see the number of training positive edges:

In [8]:
Loaded_model.pos_data["train"]

Data(x=[6614, 768], edge_index=[2, 7397], edge_label=[7397], train_mask=[6614])

#### Recommendation for a given node

In [26]:
import torch.nn.functional as F
def recommendation_by_transductive_model(A, model):
    ## Return recommedation for A in the given graph
    neg_graph = model.neg_data["all"]
    candidate = torch.zeros(neg_graph.num_edges)

    candidate = neg_graph.edge_index[1, (neg_graph.edge_index[0] == A)].cpu()
    candidate.sort()
    num_candidates = len(candidate) 
    u = torch.ones(num_candidates, dtype=torch.int) * A
    v = torch.Tensor(candidate)
    candidate_link = torch.stack((u, v))

    embedding = model(model.pos_data["train"].x.cpu(), model.pos_data["train"].edge_index.cpu())

    scores = np.zeros(candidate.max()+1) - 1e8
    
    scores[v] = F.sigmoid((embedding[u] * embedding[v]).sum(dim=1)).detach().numpy()

    rank_idx = np.argsort(scores)[::-1]
    
    pos_graph = model.pos_data["all"]
    print("Recommendation for node", A, "with title", pos_graph.title[A])
    for k in range(10):
        print(rank_idx[k], scores[rank_idx[k]], pos_graph.title[rank_idx[k]])
        
recommendation_by_transductive_model(1370, Loaded_model)

Recommendation for node 1370 with title Non-linear matrix completion
119 0.9976344108581543 R1-PCA: rotational invariant L1-norm principal component analysis for robust subspace factorization
417 0.9962921142578125 Matrix Completion via Sparse Factorization Solved by Accelerated Proximal Alternating Linearized Minimization
907 0.9961626529693604 Principal manifolds and nonlinear dimensionality reduction via tangent space alignment
761 0.9953721165657043 Efficient and Robust Feature Selection via Joint ℓ2, 1-Norms Minimization
2215 0.9948280453681946 Robust Non-Linear Matrix Factorization for Dictionary Learning, Denoising, and Clustering
824 0.9944800138473511 Convex and Semi-Nonnegative Matrix Factorizations
2741 0.9936521053314209 Polynomial Matrix Completion for Missing Data Imputation and Transductive Learning
3879 0.9924148917198181 Robust nonnegative matrix factorization using L21-norm
2267 0.9922853708267212 Online High Rank Matrix Completion
5595 0.9917367100715637 Robust tenso

In [10]:
Loaded_model.pos_data["all"].test_mask.argwhere()

tensor([[   2],
        [   7],
        [  11],
        ...,
        [6596],
        [6597],
        [6613]])

In [23]:
recommendation_by_transductive_model(1370, SAGEmodel)

Recommendation for node 1370 with title Non-linear matrix completion
119 0.9976344108581543 R1-PCA: rotational invariant L1-norm principal component analysis for robust subspace factorization
417 0.9962921142578125 Matrix Completion via Sparse Factorization Solved by Accelerated Proximal Alternating Linearized Minimization
907 0.9961626529693604 Principal manifolds and nonlinear dimensionality reduction via tangent space alignment
761 0.9953721165657043 Efficient and Robust Feature Selection via Joint ℓ2, 1-Norms Minimization
2215 0.9948280453681946 Robust Non-Linear Matrix Factorization for Dictionary Learning, Denoising, and Clustering
824 0.9944800138473511 Convex and Semi-Nonnegative Matrix Factorizations
2741 0.9936521053314209 Polynomial Matrix Completion for Missing Data Imputation and Transductive Learning
3879 0.9924148917198181 Robust nonnegative matrix factorization using L21-norm
2267 0.9922853708267212 Online High Rank Matrix Completion
5595 0.9917367100715637 Robust tenso

In [24]:
recommendation_by_transductive_model(1676,  SAGEmodel)

Recommendation for node 1676 with title A Semismooth Newton Stochastic Proximal Point Algorithm with Variance Reduction
5541 0.9937726855278015 A Proximal Alternating Direction Method of Multiplier for Linearly Constrained Nonconvex Minimization
125 0.9928070306777954 Error bounds for analytic systems and their applications
586 0.9912087917327881 An analytic center cutting plane method for pseudomonotone variational inequalities
1389 0.9911084175109863 Error bounds and convergence analysis of feasible descent methods: a general approach
814 0.9910992980003357 Complexity Analysis of an Interior Cutting Plane Method for Convex Feasibility Problems
179 0.9910261034965515 Push–Pull Gradient Methods for Distributed Optimization in Networks
6356 0.9905016422271729 On the convergence of the coordinate descent method for convex differentiable minimization
4406 0.9901732206344604 Mathematical Programs with Equilibrium Constraints
5419 0.9899898767471313 On the Superlinear and Quadratic Converge